In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time

In [ ]:
# 准备数据

df = pd.read_excel("硕士信息大表（学生版）.xlsx")
df = df.dropna()["导一"]

In [ ]:


# 配置 Selenium WebDriver (确保你已经下载并配置好 ChromeDriver 或其他浏览器驱动)
driver = webdriver.Chrome()  # 如果你用的是 Chrome 浏览器
driver.get("https://www.bing.com")

# 定义一个空列表，用来存储搜索结果地址
results = []

time.sleep(2)  # 等待页面加载

# 遍历每个名字并进行搜索
for name in df['导一']:
    search_query = f"{name} - 哈尔滨工业大学教师个人主页 - HIT"
    
    # 找到 Bing 搜索框并输入查询内容
    search_box = driver.find_element(By.NAME, "q")
    search_box.clear()
    search_box.send_keys(search_query)
    search_box.send_keys(Keys.RETURN)
    
    time.sleep(2)  # 等待页面加载
    
    # 获取第一个搜索结果的地址
    try:
        first_result = driver.find_element(By.CSS_SELECTOR, "li.b_algo h2 a")
        first_result_url = first_result.get_attribute("href")
    except Exception as e:
        first_result_url = "未找到结果"
    
    results.append(first_result_url)
    time.sleep(1)  # 等待页面加载

# 将结果添加到 DataFrame 新的一列
df['url'] = results

# 关闭浏览器
driver.quit()

# 显示带有新列的 DataFrame
print(df)


!pip install urllib3==2.1.0


In [ ]:


# 设置Chrome选项
chrome_options = Options()
chrome_options.add_argument("--headless")  # 无头模式，不打开浏览器窗口
chrome_options.add_argument("--disable-gpu")  # 如果系统不支持GPU加速，禁用它
chrome_options.add_argument("--no-sandbox")  # 必要时禁用沙箱

# 指定chromedriver的路径
service = Service()  # 请将/path/to/chromedriver替换为你的chromedriver路径
driver = webdriver.Chrome(service=service, options=chrome_options)

# 用于存储每个页面的内容
content_list = []

try:
    for index, row in df.iterrows():
        url = row["url"]
        driver.get(url)
        
        try:

            # 等待页面加载完成，最长等待时间10秒
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "part_box"))
            )

            # 获取带有class="editor_content"的div内容
            part_boxes = driver.find_elements(By.CLASS_NAME, 'part_box')
            if len(part_boxes) < 2:
                print(None)  
                content_list.append(None) # 如果没有第二个 'part_box'，返回 None
                continue

            second_part_box = part_boxes[1]

            div_element = second_part_box.find_element(By.CLASS_NAME, 'editor_content')

            # 获取文本内容，包括伪元素
            content = driver.execute_script("return window.getComputedStyle(arguments[0], '::before').getPropertyValue('content') + arguments[0].textContent + window.getComputedStyle(arguments[0], '::after').getPropertyValue('content');", div_element)

            # 将内容添加到列表
            content_list.append(content)
        except Exception as e:
            print(f"抓取 {url} 时出错：{e}")  # 如果出现错误，打印错误信息
            content_list.append(None)

finally:
    driver.quit()

# 将内容保存回DataFrame中
df['editor_content'] = content_list

# 输出结果
print(df)


In [ ]:
df

In [ ]:
df.to_excel('output.xlsx', encoding="utf-8",index=False)